# 🔄 Notebook 06: Full EcoStackML Pipeline

In [ ]:
from ecostackml.data.loader import DataLoader
df = DataLoader.from_csv("notebooks/demo_data/sample.csv")
df["target"] = [0, 0, 1, 0, 1]

In [ ]:
from ecostackml.preprocessing.cleaner import Cleaner
cleaner = Cleaner(strategy="median", scaling="standard", anomaly_method="iqr")
df_clean = cleaner.fit_transform(df)

In [ ]:
from ecostackml.data.splitter import split_data
X_train, X_test, _, y_train, y_test, _ = split_data(df_clean, target_column="target", stratify=True)

In [ ]:
from ecostackml.models.stacker import ModelStacker
stacker = ModelStacker(
    base_models_config=[{"name": "random_forest"}, {"name": "xgboost"}],
    meta_model_name="logistic", model_type="classification")
stacker.fit(X_train, y_train)

In [ ]:
y_pred = stacker.predict(X_test)
import pandas as pd
meta_input = pd.DataFrame([
    model.model.predict(X_test) for model in stacker.base_models
]).T
y_proba = stacker.meta_model.model.predict_proba(meta_input)[:, 1]

In [ ]:
from ecostackml.models.evaluator import evaluate_classification
metrics = evaluate_classification(y_test, y_pred, y_proba)
metrics

In [ ]:
stacker.explain_base_models(X_test)
stacker.explain_meta_model(X_test)

In [ ]:
from ecostackml.utils.save_load import save_model, save_predictions, save_metrics
save_model(stacker.meta_model.model, "models/stacked_meta_model.pkl")
save_predictions(y_test, y_pred, "results/predictions.csv")
save_metrics(metrics, "results/metrics.json")